In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [67]:
words = open('names.txt' , 'r').read().splitlines()
words[:3]

['emma', 'olivia', 'ava']

In [68]:
len(words)

32033

In [69]:
chars = sorted(list(set(''.join(words))))
stoi = {char : i+1 for i , char in enumerate(chars)}
stoi['.'] = 0
itos = {i : char for char , i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [70]:
block_size = 3 #context lenght : How many characters we take to predict the next character
X , Y = [] , []
for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)


In [71]:
X.shape , Y.shape , X.dtype , Y.dtype

(torch.Size([228146, 3]), torch.Size([228146]), torch.int64, torch.int64)

In [72]:
C = torch.randn((27,2))

In [73]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [74]:
W1 = torch.randn((6 , 100))
b1 = torch.randn((100))


In [75]:
h = torch.tanh(emb.view(emb.shape[0] , 6) @ W1 + b1)

In [76]:
h.shape

torch.Size([228146, 100])

In [77]:
W2 = torch.randn((100 , 27))
b2 = torch.randn((27))

logits = h @ W2 + b2


In [78]:
logits.shape

torch.Size([228146, 27])

In [79]:
counts = logits.exp()
prob = counts / counts.sum(1 , keepdim = True)

In [80]:
prob.shape

torch.Size([228146, 27])

In [81]:
X.shape , Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [82]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27 , 2) , generator = g)
W1 = torch.randn((6 , 100) , generator = g)
b1 = torch.randn((100) , generator = g)
W2 = torch.randn((100 , 27) , generator = g)
b2 = torch.randn((27) , generator = g)
parameters = [C , W1 , b1 , W2 , b2]

In [83]:
sum(p.nelement() for p in parameters) #number of parameters

3481

In [84]:
for p in parameters:
    p.requires_grad = True

In [95]:
#Forward pass
for _ in range(1000):

    #minibatch 
    ix = torch.randint(0 , X.shape[0], (32,))

    emb = C[X[ix]]
    h  = torch.tanh(emb.view(emb.shape[0] , 6) @ W1 + b1)
    logits = h @ W2 + b2
    # counts = logits.exp()
    # prob = counts / counts.sum(1, keepdim=True)
    # loss = -prob[torch.arange(32) , Y].log().mean()
    loss = F.cross_entropy(logits , Y[ix])
    for p in parameters:
        p.grad = None
    loss.backward()
    #update
    for p in parameters:
        p.data -= p.grad.data * 0.1
print(loss.item())

2.6305158138275146


In [96]:
emb = C[X]
h = torch.tanh(emb.view(emb.shape[0] , 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits , Y)
loss

tensor(2.5537, grad_fn=<NllLossBackward0>)